# Predicting Freight Value in the US - Data Cleaning
In this project, I will investigate what factors affect the freight value for freight that is transported within the United States. I have taken a dataset from the [Bureau of Transportation Statistics](https://www.bts.gov), which is part of the [US Department of Transportation](https://www.transportation.gov). The dataset will only consider data for the year of 2017. Features in the dataset include: the mode of transportation, the origin and destination, the commodity type, the trade type, the weight, the value, and the distance traveled.

In this first notebook, I will clean the dataset to make it suitable for modeling. A second notebook will be dedicated to create the model to answer this project's question.

In [1]:
import numpy as np
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy_utils import create_database, database_exists
import warnings
warnings.filterwarnings('ignore')

import helpers as hp
from config import usr, pwd, url, port, db, table

## Load Dataset
Load the dataset into a Pandas `DataFrame`.

In [2]:
df = pd.read_csv('./data/FAF4.5_2017.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1814034 entries, 0 to 1814033
Data columns (total 14 columns):
fr_orig        float64
dms_orig       int64
dms_dest       int64
fr_dest        float64
fr_inmode      float64
dms_mode       int64
fr_outmode     float64
sctg2          int64
trade_type     int64
tons_2017      float64
value_2017     float64
tmiles_2017    float64
curval_2017    float64
wgt_dist       float64
dtypes: float64(9), int64(5)
memory usage: 193.8 MB


## Split Variable Types
As seen in the print out above for `df.info()`, all variables are numeric. But, it is clear from the documentation that many of these variables are categorical. To determine which are categorical and which are continuous, I am going to print the number of unique values for each variable.

In [4]:
for column in df.columns:
    print(f'There are {len(df[column].unique())} unique values for {column}.')

There are 9 unique values for fr_orig.
There are 132 unique values for dms_orig.
There are 132 unique values for dms_dest.
There are 9 unique values for fr_dest.
There are 8 unique values for fr_inmode.
There are 8 unique values for dms_mode.
There are 8 unique values for fr_outmode.
There are 43 unique values for sctg2.
There are 3 unique values for trade_type.
There are 130082 unique values for tons_2017.
There are 205806 unique values for value_2017.
There are 112568 unique values for tmiles_2017.
There are 206688 unique values for curval_2017.
There are 680714 unique values for wgt_dist.


**Observations:** Any variable with less than 140 unique values are categorical variables. All other variables are continuous.

I will create two lists to distinguish between continuous and categorical variables.

In [5]:
continuous = []
categorical = []
for column in df.columns:
    if len(df[column].unique()) < 140:
        categorical.append(column)
    else:
        continuous.append(column)
print(f'The categorical columns are:\n {categorical}')
print(f'The continuous columns are:\n {continuous}')

The categorical columns are:
 ['fr_orig', 'dms_orig', 'dms_dest', 'fr_dest', 'fr_inmode', 'dms_mode', 'fr_outmode', 'sctg2', 'trade_type']
The continuous columns are:
 ['tons_2017', 'value_2017', 'tmiles_2017', 'curval_2017', 'wgt_dist']


## Missing Values
Let's deal with missing values for each variable type separately.

### Continuous Variables
First, I will use a custom function to examine the fraction of missing values for the continuous variables.

In [6]:
hp.find_na_columns(df.loc[:, continuous], display_fractions=True)

Variables with missing values and their fraction of missing values:
Series([], dtype: float64)


**Observations:** There does not appear to be any missing values for this variable type.

In [7]:
df.loc[:, continuous].describe().T

,count,mean,std,min,25%,50%,75%,max
tons_2017,1814034.0,9.825770,332.737854,0.0,0.00030,0.00750,0.1362,105266.0771
value_2017,1814034.0,10.113904,192.679806,0.0,0.00460,0.05830,0.8082,70962.5892
tmiles_2017,1814034.0,2.802205,95.892118,0.0,0.00020,0.00580,0.1182,43088.9030
curval_2017,1814034.0,9.647496,157.742041,0.0,0.00470,0.05900,0.8177,52077.2198
wgt_dist,1814034.0,1089.400935,879.965088,0.0,405.03655,888.23455,1535.3388,9970.0000


In [8]:
print(f'The dataset length is {len(df)}.')

The dataset length is 1814034.


**Observations:** Since the dataset length is the same as the value counts for each continuous variable, this is further corroboration that all continuous variables do not contain any missing values.

### Categorical Variables
Now, I will examine the missing value count for the categorical variables.

In [9]:
hp.find_na_columns(df.loc[:, categorical], display_fractions=True)

Variables with missing values and their fraction of missing values:
fr_orig       0.604456
fr_dest       0.632710
fr_inmode     0.604456
fr_outmode    0.632710
dtype: float64


**Observations:** A significant number of the categorical variables related to foreign freight have missing values. Let's examine the unique values for each of these variables.

In [10]:
na_fractions = hp.find_na_columns(df.loc[:, categorical])
for column in na_fractions.index.tolist():
    print(f'{column}: {np.unique(df[column].unique())}')

fr_orig: [801. 802. 803. 804. 805. 806. 807. 808.  nan]
fr_dest: [801. 802. 803. 804. 805. 806. 807. 808.  nan]
fr_inmode: [ 1.  2.  3.  4.  5.  6.  7. nan]
fr_outmode: [ 1.  2.  3.  4.  5.  6.  7. nan]


**Observations:** Origin and destination have the same values. In and out modes have the same values. All missing values are recorded using _nan_. 

I will fill missing values for the foreign related columns with zeros. Since these are categorical variables, the zeros can indicate that that category is not applicable the shipment for that row. In other words, values with zero for the foreign related columns will indicate that that shipment was only domestic.

Before filling missing values, I will copy the raw dataset to a new `DataFrame`.

In [11]:
df_clean = df.copy()

In [12]:
df_clean.fillna(0, inplace=True)

In [13]:
hp.find_na_columns(df_clean.loc[:, categorical], display_fractions=True)

Variables with missing values and their fraction of missing values:
Series([], dtype: float64)


Great! I now have dealt with all missing values.

## Save Dataset
Now, the dataset is clean and can be used for modeling. I will save this cleaned dataset to a SQL database, which I can access during the modeling phase.

In [14]:
# Create the database if it doesn't exist
db_url = f"postgresql+psycopg2://{usr}:{pwd}@{url}:{port}/{db}"
if database_exists(db_url):
    pass
else:
    create_database(db_url)

In [15]:
engine = create_engine(f"postgresql+psycopg2://{usr}:{pwd}@{url}:{port}/{db}")
df_clean.to_sql(name=table, con=engine, index=False, if_exists='replace')
engine.dispose()